In [3]:
import pandas as pd 
import numpy as np

import json

import requests 
from bs4 import BeautifulSoup

#creating all imports

In [4]:
#creating a header file to 
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'} 


# Obtaining All hyperlinks for the different shows

In [16]:
# create a new empty list to store all hyperlinks
all_hyperlink = []

# create a function to get all show links with parameters of the url and header
def get_hyperlinks(url, header):
    response_soup = requests.get(url, headers = header)
    soup = BeautifulSoup(response_soup.text, 'html.parser')

    # game tablle is the block of information for all of the shows
    shows_table = soup.find_all('div', {'class': 'di-ib clearfix'})
    
    # game is the box of data that has individual shows in the show_table
    for show in shows_table:
        
        # create a new variable to loop through to obtain the hyperlink in each block
        show_links_html = show.find_all('a', href=True)
        
        # it continues onto this whether it goes into the if statement or not
        # will look through each block in show_links_html and appends
        # that hyperlink into my list of all hyperlinks
        for show_link in show_links_html:
            if 'video' not in show_link['href']:
                all_hyperlink.append(show_link['href']+'/reviews')
                
               # print(game_link['href'])
            
    return all_hyperlink

In [17]:
#iterating through 1000 shows to obtain the hyperlink for each show
# limit is the starting number for the animes, so i initilize i as 0 and adds 50 each time 

i = 0
while i <= 1000:
    url = 'https://myanimelist.net/topanime.php?type=tv&limit={}'.format(i)        
    get_hyperlinks(url, header)
    i += 50

# Iterating through all the different shows to get each page of their reviews

In [40]:
# creating an empty list to store all the review pages for each show
all_review_hyperlinks = []


# goes through each page of the reviews from 1 - 10 and stores that page of reviews to a
# list to iterate through later on

for link in all_hyperlink:
    for i in range(1, 16):
        all_review_hyperlinks.append(link+'?p={}'.format(i))

# Creating Dictionaries each review with the title of the show, reviewer, review and score

In [103]:
# importing time to give the loop some time to rest not to create issues when pulling data from website
import time

In [104]:
# create an empty list of show_reviews
show_review_list = []

# function to get each review, reviewer and score and save it in a dictionary along with the
# title of the game
def each_review(url):
    
    # sleeping for 2 seconds 
    time.sleep(2)
    
# getting the beautiful soup parser initialized, changes as the url changes which will be 
# looped in the next block
    response_soup = requests.get(url, headers = header)
    soup = BeautifulSoup(response_soup.text, 'html.parser')
    
# try and except for each page to make sure the page exists and it does not go straight to
# a 404 error page, if it does then it will skip that page and move onto the next one
    try:

#grabs the title of the game at the top of each page
        try: 
            title = soup.find('span', {'itemprop': 'name'})('span')[0].text
        
        except (TypeError, IndexError):
            try: 
                title = soup.find('span', {'itemprop': 'name'}).text
                
            except AttributeError:
                pass
            pass
# this grabs the entire block of reviews on the page, used to loop through to grab individual
# reviews and scores
        user_reviews_list = soup.find_all('div', {'class' : 'borderDark'})
    except AttributeError:
            pass
    
    try:
        
#looping through the reviews in the block to get the user, review and score
        for x in user_reviews_list:
            try:
                user = x.find('div', {'style': 'float: left;'})('a')[1].text
                user_score = int((x.find('div', {'class': 'mb8'})('div')[2].text).split()[-1])
                user_reviews = x.find('div', {'class': 'spaceit textReadability word-break pt8 mt8'}).text

# Storing the title, user, user score and review in each iteration in a dictionary that is 
# overwritten at every new iteration
                show_review_dict = {
                    'title': title,
                    'user' : user,
                    'user_score': user_score,
                    'user_review' : user_reviews
                }

# stores the dictionary in a list each iteration                
                show_review_list.append(show_review_dict)
            except AttributeError:
                pass
            
    except UnboundLocalError:
        pass
    
    
    return show_review_list

In [105]:
# loop through the function constantly changing the url using the list of hyperlinks
# created earlier

for link in all_review_hyperlinks:
    url = link
    each_review(url)

In [106]:
# store the list of dictionaries into a dataframe

df = pd.DataFrame(show_review_list)

In [107]:
# store the dataframe as a csv file

df.to_csv('anime_reviews.csv')